# Tutorial Part 2

**Goal** Loop on the events in a file and do some plots with better quality

The operations you can do with the Draw() function are quite limited. Let's produce an invariant mass plot. Root has many libraries to do special relativity calculations (`TLorentzVector`). Also produce an efficiency plot

In [ ]:
import ROOT
from ROOT import TCanvas, TF1, TH1F, TTree


In [ ]:
%jsroot on

In [ ]:
#f = ROOT.TFile("../../data/root_session/mc_147771.Zmumu.root",""READ)
f = ROOT.TFile("~/data/root_session/mc_147771.Zmumu_10000.root","READ")
t1 = f.mini

For developing it is convenient to use the smaller file. <br>
Note. To access the variables inside the TTree you have to use
    `t1.name_of_the variable`

## Tips and Tricks
You can save not only the contents of the ntuple in root format. You can also print directly the `TCanvas` you've produced in any format you want (png, jpg, svg, pdf, root, C. etc.) with a command like:
    `c1.Print("file_name.pdf")`
    <br>
For pdf files you can print multiple plots in multiple files with:
    `c1.Print("file_name.pdf[")`
    <br>
The last `TCanvas` you want to print you have to close the sequence with:
    `cN.Print("file_name.pdf]")`
    

In [ ]:
from ROOT import TLorentzVector
c1 = ROOT.TCanvas("c1","My Canvas",400,400)

h_mumu= TH1F("h_mumu","Invariant mass",50,50,150);
N = t1.GetEntries()
#t1.SetBranchStatus("*",0)
#t1.SetBranchStatus("lep_pt",1)
#t1.SetBranchStatus("lep_eta",1)
#t1.SetBranchStatus("lep_phi",1)
#t1.SetBranchStatus("lep_E",1)
num_evt = lambda  x: x if x<10000 else 10000
print("Looping on ", num_evt(N), " entries")
for i in range(num_evt(N)):
    t1.GetEntry(i)

    for ilep in range(t1.lep_n):
        ivec = TLorentzVector()
        ivec.SetPtEtaPhiE(t1.lep_pt[ilep],t1.lep_eta[ilep],t1.lep_phi[ilep],t1.lep_E[ilep])
        for jlep in range(t1.lep_n):
            if ilep==jlep:
                continue
            jvec = TLorentzVector()
            jvec.SetPtEtaPhiE(t1.lep_pt[jlep],t1.lep_eta[jlep],t1.lep_phi[jlep],t1.lep_E[jlep])
            inv_mass = (ivec+jvec).Mag()
            h_mumu.Fill(inv_mass/1000.)



c1.cd()
h_mumu.Draw()
c1.Draw()
c1.Print("plots/my_zboson.pdf[")

It looks a Z boson! great! But....
<ul>
    <li> Are we sure we are plotting invariant mass made with same flavor leptons ? (Hint: Look at variable lep_type) <\li>
        <li> Look at the tails. Do you see something strange? What is the problem? (Hint: look at the nested loop)<\li>
            <li> As you've learned this week, after detection of the particles produced in collisions, the system has to collect these collisions (trigger). (Hint: Look at trig_M and trig_E variables. These are variables that regcord the trigger decision. 1 --> event Accepted, 0 event discarded) <\li>
                <li> Again this is a MC file. There are some (hopefully small) differences wrt data. These are taken into account with the variables "scaleFactors" <\li>
                    <\ul>

## Exercise 1

<ul>
    <li> Modify the script above, run and save the histogram to a file. <\li>
    <li> Modify the script to take in account what we've just discussed and save the histogram to a different fill.
Hint: You can fill a 1 dimensional histogram with a weight as a second argument <br>
    h_mumu.Fill(inv_mass/1000.,weight)
<\li>
<\ul>    

Now open the two files and plot the same histogram. You have two posibilities.
<ul>
<li> on different pads <\li>
<li> on the same pad to evaluate the differences <\li>
<\ul> 
<br>

    
Let's start with 1)

In [ ]:
import ROOT
%jsroot on

In [ ]:
f1 = ROOT.TFile("plots/histo_datamuon.root","READ")
f2 = ROOT.TFile("plots/histo_mczmumu.root","READ")
h1=f1.Get("h_mumu")
h2=f2.Get("h_mumu")
c2 = ROOT.TCanvas("c2","Splitted Canvas",400,400)
c2.Divide(1,2)
c2.cd(1)
h1.Draw()
c2.cd(2)
h2.Draw()
c2.Draw()
c2.Print("plots/my_zboson.pdf[")

Let's try to have nicer plots

In [ ]:
h1.SetStats(0)
h2.SetStats(0)
h1.SetLineWidth(3)
h2.SetLineWidth(3)
c2.Draw()
c2.Print("plots/my_zboson.pdf[")

Now you can try to superimpose the two histograms

In [ ]:
c3 = ROOT.TCanvas("c3","Superimposed Canvas",400,400)
h1.Draw("histo")
h2.Draw("p,same")
c3.Draw()
c3.Print("plots/my_zboson.pdf[")

Are they comparable? Let's try to normalize to the same integral. You have different options.

In [ ]:
h1.Scale(h2.Integral()/h1.Integral())
c4 = ROOT.TCanvas("c4","Superimposed Canvas Norm",400,400)
h1.Draw("histo")
h2.Draw("p,same")
c4.Draw()
c4.Print("plots/my_zboson.pdf[")

## Exercise 2

Run the modified (the one with the scale factors and trigger selections) script on the full MC file  
   ../data/root_session/mc_147771.Zmumu.root
and on the data file
    ../data/root_session/DataMuons.root
    <br>
Save the histograms in two different files and plot them superimposed.

Let's plot compare them with a ratio shown on a split panel <br>
In ROOT you NEVER copy histograms. What you do is cloning them with Clone() method 

In [ ]:
ratio = h2.Clone()
ratio.Divide(h1)
ratio.SetLineColor(ROOT.kRed)
c5 = ROOT.TCanvas("c5","Splitted panel",400,400)
pad1 = ROOT.TPad("pad1","pad1" ,0 ,0.3 ,1 ,1)
pad1.SetLogy(True)
pad1.Draw()
pad1.cd()
h2.Draw ("h")
h1.Draw("pe,same")
c5.Draw()
c5.Print("plots/my_zboson.pdf[")

Now plot the splitted panel

In [ ]:
c5.cd()
pad2= ROOT.TPad("pad2","pad2",0,0.05,1,0.3)
pad2.Draw()
pad2.cd()
ratio.Draw("pe")
c5.Draw()

Can be graphically improved
First, remove the border spacing on the bottom of the top pad

In [ ]:
pad1.SetBottomMargin(0)

Now, remove the border spacing on the top of the bottom pad. Also add a bit of extra space on the bottom of the top pad so that we have some space for later.

In [ ]:
pad2.SetTopMargin(0)
pad2.SetBottomMargin(0.25)

First, get rid of the x-axis labels on the top plot, this removing the random 103 on the far right of the plot

In [ ]:
h1.SetTitle("")
h1.GetXaxis().SetLabelSize (0)
h1.GetXaxis().SetTitleSize (0)

Then increase the size of the y-axis label, which is necessary because the pad is only 70% of the full vertical
range, so we need to increase the text size to get back to a reasonable value.

In [ ]:
h1.GetYaxis().SetTitleSize(0.05)

Now do the similar things to the bottom panel, also increasing the text sizes everywhere as this is now only
30% of the full vertical range. Note that the title offset is changed at one point to bring the label closer to the
axis (so it doesn't go off the canvas).

In [ ]:
ratio.SetTitle("")
ratio.GetXaxis().SetLabelSize(0.12)
ratio.GetXaxis().SetTitleSize(0.12)
ratio.GetYaxis().SetLabelSize(0.1)
ratio.GetYaxis().SetTitleSize(0.15)
ratio.GetYaxis().SetTitle("Data/MC")
ratio.GetYaxis().SetTitleOffset(0.3)
c5.Draw()

In [ ]:
ratio.GetYaxis().SetRangeUser(0.5,1.5)
ratio.GetYaxis().SetNdivisions(207)
line=ROOT.TLine(50,1,150,1)
line.SetLineColor(ROOT.kBlack)
line.SetLineWidth(2)
line.Draw("same")
c5.Draw()

You  could add some legend to the plot

In [ ]:
legend=ROOT.TLegend(0.7,0.6,0.85,0.75)
legend.AddEntry(h1,"MC")
legend.AddEntry(h2,"Data")
legend.SetLineWidth(0)
legend.Draw("same")
c5.Draw()

Finally, add some label.

In [ ]:
latex=ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.06)
latex.DrawText(0.7,0.83,"HASCO 2021")
latex.SetTextSize(0.04)
latex.DrawText(0.7,0.77, "Di-muon events ")
c5.Draw()
c5.Print("plots/my_zboson.pdf]")

Now let's do an efficiency plot.

## Exercise
Use the modified script above and produce a denominator histogram and a numerator histogram after some selection. Ths efficiency is the ratio between these two histograms. <br>
The selection is, for example, the fact that both leptons are "trigger matched". We want to measure the efficiency as a fucntion of the leading lepton pt <bt>
** Hint:** lepton vectors are inversly ordered with momentum. So  `t1.lep_pt[0]` should be the largest one. Denominator and numerator plots are stored in `plots/my_zboson_out.root`


In [ ]:
f = ROOT.TFile("plots/my_zboson_out.root","READ")
f.ls()

In [ ]:
c_eff = TCanvas("c_eff","Efficiency",400,400)
c_eff.Divide(1,3)
c_eff.cd(1)
h_denum = f.h_mu_pt_denum
h_denum.Draw("pe")
c_eff.cd(2)
h_num = f.h_mu_pt_num
h_num.Draw("pe")
c_eff.cd(3)
h_eff = h_num.Clone()
h_eff.Divide(h_denum)
h_eff.Draw("pe")
c_eff.cd()
c_eff.Draw()

What's wrong with the efficiency plot?

The best way is to fill `TEfficiency` while looping on the events [TEfficiency](https://root.cern.ch/doc/master/classTEfficiency.html). <br>

Another (less correct way) is to use `TGraphAsymmErrors` [TGraphAsymmErrors](https://root.cern.ch/doc/master/classTGraphAsymmErrors.html) or construct [TEfficiency](https://root.cern.ch/doc/master/classTEfficiency.html) with numerator and denumerator histograms <br> 

In [ ]:
c_eff2 = TCanvas("c_eff2","Efficiency TGA",400,400)
c_eff2.Divide(1,3)
c_eff2.cd(1)
h_denum = f.h_mu_pt_denum
h_denum.Draw("pe")
c_eff2.cd(2)
h_num = f.h_mu_pt_num
h_num.Draw("pe")
c_eff2.cd(3)

tga_eff = ROOT.TGraphAsymmErrors(h_num,h_denum)
tga_eff.Draw("pe")
c_eff2.cd()
c_eff2.Draw()

In [ ]:
c_eff3 = TCanvas("c_eff3","Efficiency TEff",400,400)
c_eff3.Divide(1,3)
c_eff3.cd(1)
h_denum = f.h_mu_pt_denum
h_denum.Draw("pe")
c_eff3.cd(2)
h_num = f.h_mu_pt_num
h_num.Draw("pe")
c_eff3.cd(3)

eff_h = ROOT.TEfficiency(h_num,h_denum)
eff_h.Draw("ap")
c_eff3.cd()
c_eff3.Draw()